In [1]:
import numpy as np
# import keras 
import msprime as msp

In [2]:
# global settings
Ne = 1e4

In [ ]:
# function for converting to ms format.
def to_mksample(file, hap, pos):
    with open(file, 'w') as out:
        out.write('ms test\n')
        out.write('1 2 3\n')
        out.write('\n')
        out.write('segsites: {0}\n'.format(hap.shape[0]))
        out.write('positions: ')
        for p in pos:
            out.write(str(np.round(p/pos[-1], 4)))
            out.write(' ')
        out.write('\n')
        for n in range(hap.shape[1]):
            h = hap[:, n].tolist()
            hstr = [str(val) for val in h]
            st = ''.join(hstr)
            out.write(st)
            out.write('\n')        

In [ ]:
to_mksample('test.txt', hap, pos)

In [3]:
# generate haplotypes and positions using msprime based on some inital settings.
def generate_sample(r):
    recombination_rate=r
    mutation_rate = 2e-8
    length = 1e4
    Ne = 10000
    sample_size = 20
    tree_seqs = msp.simulate(sample_size=sample_size, Ne=Ne, length=length, recombination_rate=recombination_rate, mutation_rate=mutation_rate)
    pos = []
    for v in tree_seqs.variants():
        pos.append(v.position)
    haplotypes = tree_seqs.genotype_matrix()
    return haplotypes, pos

In [5]:
# This is the funtion to compute likelihood for a haplotype with specified order.
def loglikelihood_2(haplotypes, pos, r):
    def _theta(sample_size):
        s = 0
        for t in range(1, sample_size):
            s += 1/t
        return 1/s
    num_snps, sample_size = haplotypes.shape
#     print('num_snps', num_snps, 'sample_size', sample_size)
    log_conditional_haps = [-num_snps*np.log(2)]
    theta = _theta(sample_size)
#     theta = 8e-4
    lens = np.array(pos[1:]) - np.array(pos[:-1])
    
    for i in range(1, sample_size):
        h = haplotypes[:, i]
        a = []
        alpha= []
        # compute for a(0)
        for j in range(i):
            h_j = haplotypes[:, j]
            if h_j[0] == h[0]:
                gamma = i/(i+theta) + 1/2*theta/(i+theta)
            else:
                gamma = 1/2*theta/(i+theta)
            a.append(gamma*1/i)
        alpha.append(a.copy())
        # compute for a(1),a(2),...,a(num_snps)
        for s in range(1, num_snps):
            a = []
            for j in range(i):
                h_j = haplotypes[:, j]
                if h_j[s] == h[s]:
                    gamma = i/(i+theta) + 1/2*theta/(i+theta)
                else:
                    gamma = 1/2*theta/(i+theta)
                p = np.exp(-4*Ne*r*lens[s-1]/i)
                res = gamma * (p*alpha[-1][j] + (1-p)/i*sum(alpha[-1]))
                a.append(res)
            alpha.append(a.copy())
        log_conditional_haps.append(np.log(sum(alpha[-1])))
    return sum(log_conditional_haps)
        
                    

In [6]:
# golden bisection search for point estimation.
def golden_bisection_search(haplotypes, pos, start, end, maxiters=500):
    i = 0
    p = 0.618
    while (end-start > 1e-10) and (i < maxiters):
        left = start + (end-start) * (1-p)
        right = start + (end-start) * p
        if shuffle_sum(haplotypes, pos, left) >= shuffle_sum(haplotypes, pos, right):
            end = right
        else:
            start = left
        print('%d start: %s, end: %s'%(i, start, end))
        i += 1
    return (start+end)/2

In [7]:
# randomly choose 20 different orders for computing likelihood. It's important!
def shuffle_sum(haplotypes, pos, r):
    logs = [loglikelihood_2(haplotypes, pos, r)]
    for i in range(20):
        shuffled_hap = haplotypes.copy().T
        np.random.shuffle(shuffled_hap)
        shuffled_hap = shuffled_hap.T
        logs.append(loglikelihood_2(shuffled_hap, pos, r))
    return np.mean(logs)

In [13]:
# example for using generate_sample.
hap, pos = generate_sample(r=3.2e-8)
# for r in [1e-2, 1e-3, 5e-4, 1e-4, 5e-5, 1e-5, 5e-6, 1e-6, 5e-7, 1e-7, 6e-8, 1e-8, 5e-9, 1e-9]:
#         print(r, shuffle_sum(hap, pos, r))

In [14]:
golden_bisection_search(hap, pos, 1e-10, 1e-6)

0 start: 1e-10, end: 6.180381999999999e-07
1 start: 1e-10, end: 3.8198580759999995e-07
2 start: 1e-10, end: 2.3610542909679996e-07
3 start: 1e-10, end: 1.4595135518182237e-07
4 start: 1e-10, end: 9.023613750236621e-08
5 start: 1e-10, end: 5.5804132976462325e-08
6 start: 2.137897879700861e-08, end: 5.5804132976462325e-08
7 start: 2.137897879700861e-08, end: 4.2653724079911005e-08
8 start: 2.137897879700861e-08, end: 3.4526771381842286e-08
9 start: 2.6401435564415073e-08, end: 3.4526771381842286e-08
10 start: 2.6401435564415073e-08, end: 3.142289309958509e-08
11 start: 2.6401435564415073e-08, end: 2.9504696321150144e-08
12 start: 2.6401435564415073e-08, end: 2.8319250712077346e-08
13 start: 2.713404095082206e-08, end: 2.8319250712077346e-08
14 start: 2.713404095082206e-08, end: 2.7866500583277826e-08
15 start: 2.713404095082206e-08, end: 2.7586701003679724e-08
16 start: 2.713404095082206e-08, end: 2.7413784863488098e-08
17 start: 2.713404095082206e-08, end: 2.7306922688849673e-08
18 star

2.716705479327929e-08

In [ ]:
# test on some random data. there are totally 100 data and it does need some time to run that. I won't provide any result below.
# and some plots and measurements are based on this prediction, which couldn't be given at this time. You could run it if you are interested in that.
np.random.seed(1996)
rs = np.random.rand(100) * 1e-6
def test_accuracy(rs):
    ests = []
    for r in rs:
        hap, pos = generate_sample(r)
        est = golden_bisection_search(hap, pos, 1e-10, 1e-6)
        ests.append(est)
        print(i, np.log(est/r))
    return ests
ests = test_accuracy(rs)

In [ ]:
# boxplot analysis for log errors.
ests = np.array(ests)
logratios = np.log10(ests/rs)
import matplotlib.pyplot as plt 
plt.title(r'$\log \frac{\rho_{pred}}{\rho_{true}}$')
plt.boxplot(logratios)
plt.show()

In [ ]:
# test MAE with groundtruth.
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(rs, ests)
mae

In [ ]:
# plot log errors as a distribution.
import seaborn as sns
plt.title(r'kdeplot of $log_{10}\frac{r_{est}}{r_{true}}$')
sns.distplot(logratios, bins=20)
plt.xlabel('log(error)')
plt.ylabel('density')
plt.savefig('logerror', dpi=200)

### Test for pytorch (always get Nan during computing gradients, I plan to work with that later.)

In [ ]:
def loglikelihood_cuda(haplotypes, pos, r):
    def _theta(sample_size):
        s = 0
        for t in range(1, sample_size):
            s += 1/t
        return 1/s
    num_snps, sample_size = haplotypes.shape
    print('num_snps', num_snps, 'sample_size', sample_size)
    log_conditional_haps = [-num_snps*np.log(2)]
    theta = _theta(sample_size)
#     theta = 8e-4
    lens = np.array(pos[1:]) - np.array(pos[:-1])
    
    for i in range(1, sample_size):
        h = haplotypes[:, i]
        px = [1/sample_size]
        a = []
        alpha= []
        # compute for a(0)
        for j in range(i):
            h_j = haplotypes[:, j]
            if h_j[0] == h[0]:
                gamma = i/(i+theta) + 1/2*theta/(i+theta)
            else:
                gamma = 1/2*theta/(i+theta)
            a.append(gamma*1/i)
        alpha.append(a.copy())
        # compute for a(1),a(2),...,a(num_snps)
        for s in range(1, num_snps):
            a = []
            for j in range(i):
                h_j = haplotypes[:, j]
                if h_j[s] == h[s]:
                    gamma = i/(i+theta) + 1/2*theta/(i+theta)
                else:
                    gamma = 1/2*theta/(i+theta)
                p = torch.exp(-r*lens[s-1]/i)
#                 print(gamma)
                res = gamma * (p*alpha[-1][j] + (1-p)/i*sum(alpha[-1]))
                a.append(res)
            alpha.append(a.copy())
        log_conditional_haps.append(torch.log(sum(alpha[-1])))
    return -sum(log_conditional_haps)
        
                   

In [ ]:
import torch 
torch.autograd.set_detect_anomaly(False)
learning_rate=1e-8
device = torch.device('cuda:0')
r = torch.tensor(1e-6, device=device, dtype=torch.float, requires_grad=True)
for i in range(200):
    print(i)
    pred = loglikelihood_cuda(haplotypes, pos, r)
    pred.backward()
    with torch.no_grad():
        r -= learning_rate* r.grad
        r.grad = None
    print(r.grad)

In [ ]:
r = torch.tensor(1e-4, dtype=torch.float64, requires_grad=True)
y = torch.exp(-r*100/10)
y.backward()
print(r.grad)
print(np.exp(-10*r.item())*(-10))

In [ ]:
a = torch.randn((), device=device, dtype=torch.float, requires_grad=True)
b = [a,2*a,a]
c = b.copy()
y = torch.exp(sum(b))
y.backward()
print(y, a.grad)